In [1]:
#load in required packages
import mediapipe as mp #mediapipe
import cv2 #opencv
import math #basic operations
import numpy as np #basic operations
import pandas as pd #data wrangling
import csv #csv saving
import os #some basic functions for inspecting folder structure etc.

#list all videos in input_videofolder
from os import listdir
from os.path import isfile, join
mypath = "./input_videos/" #this is your folder with (all) your video(s)
#time series output folder
inputfol = "./input_videos/"
outputf_mask = "./Output_Videos/"
outtputf_ts = "./Output_TimeSeries/"
# create output folders if they do not exist
if not os.path.exists(outputf_mask):
    os.makedirs(outputf_mask)
if not os.path.exists(outtputf_ts):
    os.makedirs(outtputf_ts)

In [2]:
import glob
# This will return a list of all .mp4 paths matching the pattern
vfiles = glob.glob(
    "./input_videos/*.mp4"
)
print(vfiles)

['./input_videos/zHELICOPTER.mp4', './input_videos/zBICYCLE_FIETS.mp4', './input_videos/S12B.mp4']


In [3]:
kp = []
mp_to_coco_body = [0, 2, 5, 7, 8, 11, 12, 13, 14, 15, 16, 23, 24, 25, 26, 27, 28]
mp_to_coco_foot = [
    31, 31, 29,   # left_big_toe, left_small_toe (dup), left_heel
    32, 32, 30    # right_big_toe, right_small_toe (dup), right_heel
]
from mp2dlib import convert_landmarks_mediapipe_to_dlib
# results.face_landmarks is the MP FaceMesh output


In [4]:
import cv2
import mediapipe as mp

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

def draw_and_save_custom_landmarks(image, results, skip_pose_ids=None,
                          hand_landmark_style=None,
                          hand_connection_style=None,
                          pose_point_radius=4,
                          pose_point_color=(0,255,0),
                          pose_connection_style=None,
                          connect_hands_to_body=True,
                          arm_connection_color=(255,0,0),
                          arm_connection_thickness=2, 
                          draw=False):
    h, w, _ = image.shape
    """
    Draws all hand landmarks + filtered pose landmarks on `image`.

    Args:
      image:       BGR image to draw onto.
      results:     Holistic.process(...) results.
      skip_pose_ids: set of mp_holistic.PoseLandmark to omit.
      hand_landmark_style, hand_connection_style:
        DrawingSpec for hands (defaults to MP styles).
      pose_point_radius, pose_point_color:
        circle style for filtered pose points.
      pose_connection_style:
        DrawingSpec for pose connections (defaults to green, thickness=2).
    """
    skip_pose_ids = skip_pose_ids or set()
    # default styles
    hand_landmark_style    = hand_landmark_style    or mp_styles.get_default_hand_landmarks_style()
    hand_connection_style  = hand_connection_style  or mp_styles.get_default_hand_connections_style()
    pose_connection_style  = pose_connection_style  or mp_drawing.DrawingSpec(color=pose_point_color, thickness=2)

    # 1) draw **all** hand landmarks
    frame_keypoints = []
    if results.left_hand_landmarks:
        if draw:
            mp_drawing.draw_landmarks(
                image,
                results.left_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=hand_landmark_style,
                connection_drawing_spec=hand_connection_style
            )
        # add left hand keypoints to frame_keypoints
        for idx, lm in enumerate(results.left_hand_landmarks.landmark):
            frame_keypoints.append([lm.x, lm.y, lm.z, lm.visibility])
    else:
        # If no left hand landmarks, add placeholders
        for i in range(21):
            frame_keypoints.append([0, 0, 0, 0])
    
    if results.right_hand_landmarks:
        if draw:
            mp_drawing.draw_landmarks(
                image,
                results.right_hand_landmarks,
                mp_holistic.HAND_CONNECTIONS,
                landmark_drawing_spec=hand_landmark_style,
                connection_drawing_spec=hand_connection_style
            )
        # add right hand keypoints to frame_keypoints
        for idx, lm in enumerate(results.right_hand_landmarks.landmark):
            frame_keypoints.append([lm.x, lm.y, lm.z, lm.visibility])
    else:
        # If no right hand landmarks, add placeholders
        for i in range(21):
            frame_keypoints.append([0, 0, 0, 0])
    # 2) draw **filtered** pose points & connections
    if results.pose_landmarks:
        h, w, _ = image.shape
        if draw:
            # draw the points (skip any in skip_pose_ids)
            for idx, lm in enumerate(results.pose_landmarks.landmark):
                landmark = mp_holistic.PoseLandmark(idx)
                if landmark in skip_pose_ids:
                    continue
                x, y = int(lm.x * w), int(lm.y * h)
                cv2.circle(image, (x, y), pose_point_radius, pose_point_color, -1)
            

        
        
        # add filtered connections to frame_keypoints
        for idx, lm in enumerate(results.pose_landmarks.landmark):
            if idx in skip_pose_ids:
                continue
            else:
                frame_keypoints.append([lm.x, lm.y, lm.z, lm.visibility])
            

        # draw connections
        if draw:
            # build filtered connections
            filtered_conns = [
                (start, end)
                for (start, end) in mp_holistic.POSE_CONNECTIONS
                if (mp_holistic.PoseLandmark(start) not in skip_pose_ids and
                    mp_holistic.PoseLandmark(end  ) not in skip_pose_ids)
            ]
            mp_drawing.draw_landmarks(
                image,
                results.pose_landmarks,
                filtered_conns,
                landmark_drawing_spec=None,            # already drew circles
                connection_drawing_spec=pose_connection_style
            )
            # 3) optionally connect each hand’s wrist back to its elbow
            if connect_hands_to_body and results.pose_landmarks:
                # LEFT
                if results.left_hand_landmarks:
                    l_wrist = results.left_hand_landmarks.landmark[0]
                    l_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.LEFT_ELBOW]
                    p1 = (int(l_wrist.x * w), int(l_wrist.y * h))
                    p2 = (int(l_elbow.x * w), int(l_elbow.y * h))
                    cv2.line(image, p1, p2, arm_connection_color, arm_connection_thickness)

                # RIGHT
                if results.right_hand_landmarks:
                    r_wrist = results.right_hand_landmarks.landmark[0]
                    r_elbow = results.pose_landmarks.landmark[mp_holistic.PoseLandmark.RIGHT_ELBOW]
                    p1 = (int(r_wrist.x * w), int(r_wrist.y * h))
                    p2 = (int(r_elbow.x * w), int(r_elbow.y * h))
                    cv2.line(image, p1, p2, arm_connection_color, arm_connection_thickness)
    else:   
        # If no pose landmarks, add placeholders
        for i in range(len(costume_markers)):
            frame_keypoints.append([0, 0, 0, 0])

    return image, frame_keypoints


In [8]:
import cv2
import mediapipe as mp
from tqdm import tqdm

mp_holistic = mp.solutions.holistic
mp_drawing = mp.solutions.drawing_utils
mp_styles = mp.solutions.drawing_styles

# Which PoseLandmark indices we want to skip because
SKIP_POSE_IDS = {
    mp_holistic.PoseLandmark.LEFT_WRIST,
    mp_holistic.PoseLandmark.RIGHT_WRIST,
    mp_holistic.PoseLandmark.LEFT_PINKY,
    mp_holistic.PoseLandmark.RIGHT_PINKY,
    mp_holistic.PoseLandmark.LEFT_INDEX,
    mp_holistic.PoseLandmark.RIGHT_INDEX,
    mp_holistic.PoseLandmark.LEFT_THUMB,
    mp_holistic.PoseLandmark.RIGHT_THUMB,
    mp_holistic.PoseLandmark.LEFT_HIP,
    mp_holistic.PoseLandmark.RIGHT_HIP,
    mp_holistic.PoseLandmark.LEFT_KNEE,
    mp_holistic.PoseLandmark.RIGHT_KNEE,
    mp_holistic.PoseLandmark.LEFT_ANKLE,
    mp_holistic.PoseLandmark.RIGHT_ANKLE,
    mp_holistic.PoseLandmark.LEFT_HEEL,
    mp_holistic.PoseLandmark.RIGHT_HEEL,
    mp_holistic.PoseLandmark.LEFT_FOOT_INDEX,
    mp_holistic.PoseLandmark.RIGHT_FOOT_INDEX,
    mp_holistic.PoseLandmark.NOSE,
    mp_holistic.PoseLandmark.LEFT_EYE_INNER,
    mp_holistic.PoseLandmark.LEFT_EYE,
    mp_holistic.PoseLandmark.LEFT_EYE_OUTER,
    mp_holistic.PoseLandmark.RIGHT_EYE_OUTER,
    mp_holistic.PoseLandmark.RIGHT_EYE,
    mp_holistic.PoseLandmark.RIGHT_EYE_INNER,
    mp_holistic.PoseLandmark.RIGHT_EYE_OUTER,
    mp_holistic.PoseLandmark.LEFT_EAR,
    mp_holistic.PoseLandmark.RIGHT_EAR,
    mp_holistic.PoseLandmark.MOUTH_LEFT,
    mp_holistic.PoseLandmark.MOUTH_RIGHT, 
}
# Process videos
for vidf in vfiles:
    print(f"Processing video: {vidf}")
    print(f"Video {vfiles.index(vidf)+1} of {len(vfiles)}")
    
    videoname = vidf
    videoloc = videoname
    capture = cv2.VideoCapture(videoloc)
    # get the number of frames in the video
    frameWidth = capture.get(cv2.CAP_PROP_FRAME_WIDTH)
    frameHeight = capture.get(cv2.CAP_PROP_FRAME_HEIGHT)
    samplerate = capture.get(cv2.CAP_PROP_FPS)
    # get the number of frames in the video
    num_frames = int(capture.get(cv2.CAP_PROP_FRAME_COUNT))
    # get the number of frames in the video
    print(f"Number of frames in the video: {num_frames}")
    with mp_holistic.Holistic(static_image_mode=False,           # Video stream mode :contentReference[oaicite:7]{index=7}
    model_complexity=1,                # Highest-accuracy pose model :contentReference[oaicite:8]{index=8}
    refine_face_landmarks=False,        # Finer facial detail (iris, contours) :contentReference[oaicite:9]{index=9}
    enable_segmentation=False,          # Person mask for effects :contentReference[oaicite:10]{index=10}
    smooth_landmarks=True,             # Temporal smoothing to reduce jitter :contentReference[oaicite:11]{index=11}
    min_detection_confidence=0.7,      # Filter weak detections :contentReference[oaicite:12]{index=12}
    min_tracking_confidence=0.7        # Filter unstable tracks :contentReference[oaicite:13]{index=13}
    ) as holistic:
        all_kpts = []
        for i in tqdm(range(num_frames), desc="Processing frames", unit="frame"):
            ret, frame = capture.read()
            if not ret:
                break
            image = cv2.cvtColor(frame, cv2.COLOR_BGR2RGB)
            results = holistic.process(image)
            h, w, _ = image.shape
            image = cv2.cvtColor(image, cv2.COLOR_RGB2BGR)
            image, kpts = draw_and_save_custom_landmarks(
                    image,
                    results,
                    skip_pose_ids=SKIP_POSE_IDS,
                    pose_point_radius=5,
                    pose_point_color=(0,255,0),
                    connect_hands_to_body=True,
                    arm_connection_color=(255,0,0),       # red lines for the “arm” link
                    arm_connection_thickness=2,
                    draw=True
                )
            all_kpts.append(kpts)

            cv2.imshow("merged_landmarks", image)

            if cv2.waitKey(1) == 27:
               break
            cv2.waitKey(1)
        capture.release()
        cv2.destroyAllWindows()
    all_kpts = np.array(all_kpts)
    # Save the keypoints as npy array
    video_name = vidf.split('/')[-1].split('.')[0]
    np.save(outtputf_ts + video_name+ '_all_kpts_17.npy', all_kpts)
    # 

I0000 00:00:1749735315.799141 15448406 gl_context.cc:369] GL version: 2.1 (2.1 Metal - 88.1), renderer: Apple M3 Pro


Processing video: ./input_videos/zHELICOPTER.mp4
Video 1 of 3
Number of frames in the video: 142


Processing frames:   0%|          | 0/142 [00:00<?, ?frame/s]W0000 00:00:1749735315.860851 15522911 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735315.870455 15522911 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735315.871588 15522909 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735315.871622 15522918 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735315.871629 15522913 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735315.8

Processing video: ./input_videos/zBICYCLE_FIETS.mp4
Video 2 of 3
Number of frames in the video: 131


Processing frames:   0%|          | 0/131 [00:00<?, ?frame/s]W0000 00:00:1749735323.270132 15523022 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735323.279173 15523027 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735323.280258 15523027 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735323.280313 15523024 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735323.280335 15523025 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735323.2

Processing video: ./input_videos/S12B.mp4
Video 3 of 3
Number of frames in the video: 17940


Processing frames:   0%|          | 0/17940 [00:00<?, ?frame/s]W0000 00:00:1749735330.128912 15523099 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735330.138766 15523100 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735330.139852 15523098 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735330.139953 15523106 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735330.139960 15523097 inference_feedback_manager.cc:114] Feedback manager requires a model with a single signature inference. Disabling support for feedback tensors.
W0000 00:00:1749735330

KeyboardInterrupt: 